In [125]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy.stats import multivariate_normal
from astropy.visualization import ZScaleInterval
from scipy.signal import convolve2d
interval = ZScaleInterval()

In [1]:
from photutils.detection import StarFinder

- peak pixel rate of $p \otimes D$ (p is a Gaussian with standard detivation $\sigma_g$, D is a white noise image, $\nu$ is the SNR.)


$n(\nu) = \frac{1}{2^{5/2}\pi^{3/2}\sigma_g^2} \nu e^{-\nu^2/2}$

In [2]:
def cal_n(nu, sg):
    n = 1 / 2**(5/2) / np.pi**(3/2) / sg**2 * nu * np.exp(- nu**2 / 2)
    return n

def compute_xy_grids(x_len, y_len):
    x = np.arange(- x_len // 2 + 1, x_len // 2 + 1, 1)
    y = np.arange(- y_len // 2 + 1, y_len // 2 + 1, 1)
    xx, yy = np.meshgrid(x, y)
    return xx, yy
    
def gaussian2d(xx, yy, m=[0., 0.], cov=[[1, 0], [0, 1]]):
    grid = np.dstack((xx, yy))
    var = multivariate_normal(mean=m, cov=cov)
    return var.pdf(grid)

def sim_score_image(image_size, sky_sig, psf_sig, random_seed=None):
    if random_seed:
        np.random.seed(random_seed)
    image =  np.random.normal(scale=sky_sig, size=(image_size, image_size))

    psf_xx, psf_yy = compute_xy_grids(image_size, image_size)
    psf = gaussian2d(
            psf_xx, psf_yy, m=[0, 0], cov=[[psf_sig, 0], [0, psf_sig]]
        )
    # score = scipy.signal.fftconvolve(image, psf, mode='same')
    score = convolve2d(image, psf, mode='same')
    return psf, image, score

In [ ]:
image_size = 2001
sky_sig = 1
psf_sig = 3
psf, image, score = sim_score_image(image_size, sky_sig, psf_sig, random_seed=1)

In [ ]:
np.std(score)

In [ ]:
plt.imshow(interval(score), cmap='gray')

In [116]:
threshold = 2

In [117]:
n_theory = cal_n(threshold, psf_sig)

In [118]:
peaks_theory = n_theory * image_size * image_size

In [119]:
peaks_theory

3822.8996801413577

In [120]:
def find_peaks(image, nu):
    n = (image > nu).sum()
    return n

In [121]:
find_peaks(score, threshold)

0

In [123]:
score.max()

0.7814526897050111

In [124]:
fwhm = 2.355 * sky_sig
5.8 * 1e-3 / fwhm**2

0.0010457940597094315

In [105]:
cal_n(3, psf_sig)

0.00011755843029898338